##### Note on running the notebook (solved)

Since the graph is written in the default graph, it is better to reset it using:

```
tf.reset_default_graph()
```

~~What is currently a bit strange, is that this code only works once. If you want to let this notebook run for the second time, you seem to have to restart the notebook (Kernel->restart). This is not the case if you put all the code into a python file. ~~


### Looking in the detail of how TensorFlow opimizes the problem
The computation of residual sum of error of the linear regression
$$
    y = a x + b 
$$

is encoded in the follwong TensorFlow graph, which reads in x_data and y_data and does some processing. The nodes of the graph, are operations called *ops*. For the documemation: "An op
takes zero or more `Tensors`, performs some computation, and produces zero or
more `Tensors`.  A `Tensor` is a typed multi-dimensional array." In our example, the Tensor `x_data` is a simple vector of size N.


In this notebook we investigate, who to write such a graph and also include additional information, while the graph is processed.
![graph.png, width=140](graph.png)

In [1]:
# Creating the data
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
N = 30
x_vals = (np.linspace(0,10,N)).astype('float32')
y_vals = (2.42 * x_vals + 0.42 + np.random.normal(0,1,N)).astype('float32')

In [5]:
tf.reset_default_graph()
a = tf.Variable(1.0, name = 'a') #Note that 1.0 is needed
b = tf.Variable(0.01, name = 'b')
x = tf.placeholder('float32', [N], name='x_data')
y = tf.placeholder('float32', [N], name='y_data')


resi = a*x + b - y
loss = tf.reduce_sum(tf.square(resi), name='loss')
init_op = tf.initialize_all_variables() #Initialization op 

train_op = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)
epochs = 1000

loss_summary = tf.scalar_summary("loss_summary", loss) #<--- Definition of ops to be stored
resi_summart = tf.histogram_summary("resi_summart", resi)
merged_summary_op = tf.merge_all_summaries()        #<-----  Combine all ops to be stored
sess = tf.Session()
sess.run(init_op)
writer = tf.train.SummaryWriter("/tmp/dumm/run1", tf.get_default_graph(), 'graph.pbtxt') #<--- Where to store
for e in range(epochs): #Fitting the data for 10 epochs
    sess.run(train_op, feed_dict={x:x_vals, y:y_vals})
    if (e < 5 | e > epochs - 5):
        print("epoch {} {}".format(e, sess.run(loss, feed_dict={x:x_vals, y:y_vals})))
    sum_str = sess.run(merged_summary_op, feed_dict={x:x_vals, y:y_vals}) #<--- Running the graph to produce output
    writer.add_summary(sum_str, e) #<--- writing out the output
res = sess.run([loss, a, b], feed_dict={x:x_vals, y:y_vals})
print(res)
print('Finished all')

epoch 992 16.5880947113
epoch 993 16.5874900818
epoch 994 16.5868778229
epoch 995 16.5862636566
epoch 996 16.5856647491
epoch 998 16.5844516754
[16.583851, 2.3717301, 0.80445063]
Finished all


The graph is created and can be visualized using:
```
    tensorboard --logdir=/tmp/dumm
```
and explored using the browser with the URL: http://0.0.0.0:6006, you will find something like:
![graph.png](tf_screenshot.png)

## Getting hold of tensors

Suppose you have not access to the graph construction but would like to feed and fetch the graph later for e.g. for debugging. You could do to following top get all necessary tensors for debugging

In [3]:
loss_t = tf.Graph.get_tensor_by_name(tf.get_default_graph(), 'loss:0')
x_1 = tf.Graph.get_tensor_by_name(tf.get_default_graph(), 'x_data:0')
y_1 = tf.Graph.get_tensor_by_name(tf.get_default_graph(), 'y_data:0')
loss_val = sess.run(loss_t, feed_dict={x_1:x_vals, y_1:y_vals})
print(loss_val)

16.5839


In [4]:
sess.close()